In [1]:
import keras.backend as K
from keras.layers import Dense, Activation, Flatten, Dropout, Conv2D
from keras.layers import AveragePooling2D, concatenate, Input, Lambda, Cropping2D, GlobalAveragePooling2D, MaxPooling2D, ZeroPadding2D
from keras.layers import concatenate, Reshape
from keras import utils
from keras.callbacks import Callback, LambdaCallback, EarlyStopping, ModelCheckpoint, TensorBoard, LearningRateScheduler, ReduceLROnPlateau
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

from keras.layers.normalization import BatchNormalization
from keras.applications.mobilenet import MobileNet, preprocess_input
import numpy as np
import pandas as pd
import cv2
import csv
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

import glob

Using TensorFlow backend.


In [2]:
# here, I am basing my network off the previous vgglike model, 
# but the output of conv block1 is also fed into the fully connected layer
def mscnn(n_classes,input_shape, ksize=(3,3), dropout=0.25):
    inputs = Input(shape=input_shape)
    block1 = Conv2D(32, kernel_size=ksize, activation='relu', padding='same', name='set1_conv1')(inputs)
    block1 = BatchNormalization()(block1)
    block1 = Conv2D(32, kernel_size=ksize, activation='relu', padding='same', name='set1_conv2')(block1)
    block1 = BatchNormalization()(block1)
    block1 = MaxPooling2D(pool_size=(2, 2),strides=(2, 2), name='set1_pool')(block1)
    
    block2 = Conv2D(64, kernel_size=ksize, activation='relu', padding='same', name='set2_conv1')(block1)
    block2 = BatchNormalization()(block2)
    block2 = Conv2D(64, kernel_size=ksize, activation='relu', padding='same', name='set2_conv2')(block2)
    block2 = BatchNormalization()(block2)
    block2 = Conv2D(64, kernel_size=ksize, activation='relu', padding='same', name='set2_conv3')(block2)
    block2 = BatchNormalization()(block2)
    block2 = MaxPooling2D(pool_size=(2, 2), strides=(2, 2),name='set2_pool')(block2)
    output2 = Flatten()(block2)
    
    output1 = Flatten()(block1)
    
    merged = concatenate([output1, output2])
    
    fcblock = Dense(1024, activation='relu', name='fc1')(merged)
    fcblock = BatchNormalization()(fcblock)
    fcblock = Dropout(dropout)(fcblock)
    fcblock = Dense(512, activation='relu' , name='fc2')(fcblock)
    fcblock = BatchNormalization()(fcblock)
    fcblock = Dropout(dropout)(fcblock)  
    predictions = Dense(n_classes, activation='softmax', name='final')(fcblock)
    model = Model(inputs=inputs, outputs=predictions)    
    return model

In [3]:
X = np.load('traindata_x_small.npy')
y = np.load('traindata_y_small.npy')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)

In [4]:
y_train = utils.to_categorical(y_train, 2)
y_test = utils.to_categorical(y_test, 2)

In [6]:
datagen =ImageDataGenerator(
    rescale=1. / 255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

In [7]:
# also define custom keras callback as verbose = 1 crashed the browser
def logger(epoch, logs):
    if epoch %2== 0: 
        print(epoch, logs['loss'], logs['acc'], logs['val_acc'])
logging_callback = LambdaCallback(
    on_epoch_end=logger)

In [17]:
K.clear_session()
model_mscnn = mscnn(2)
model_mscnn.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [49]:
cars = shuffle(cars)
notcars= shuffle(notcars)
cars_data = [img_to_array(load_img(img_path, target_size=(224, 224))) for img_path in cars]
notcars_data = [img_to_array(load_img(img_path, target_size=(224, 224))) for img_path in notcars]

In [51]:
cars_data[0].shape

(224, 224, 3)

In [56]:
X = np.array(cars_data+notcars_data)
y = np.concatenate((np.ones(len(cars)), np.zeros(len(notcars))))

In [57]:
np.save('traindata_x.npy', X)
np.save('traindata_y.npy', y)

In [54]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)

In [55]:
X_train[10]

'data/non-vehicles/GTI/image1852.png'

In [44]:
y_train[:10]

array([ 0.,  1.,  1.,  1.,  0.,  0.,  0.,  0.,  1.,  0.])

In [12]:
datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

In [16]:
model = MobileNet(weights = "imagenet",include_top=False,input_shape = (224, 224, 3))

17227776/17225924 [==============================] - 9s 1us/step


In [45]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (Activation)      (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128       
_________________________________________________________________
conv_dw_1_relu (Activation)  (None, 112, 112, 32)      0         
__________

In [ ]:
last_conv = model.get_layer('conv_pw_13_relu').output
flatten = Flatten(last_conv)
fc1 = Dense(flatten)
fc2 = Dense(fc1)
prediction = Dense(output_dim=1, activation='sigmoid', name='logit')(fc2)
model_modified = Model(input=model.input, output=prediction)
model_modified.summary()

In [ ]:
# generator for producing training/validation data in batches so there is no need to hold all of
#  the training data in memory-- given the sample size, the machine would actually run out of memory if I use
# all three training datasets (sample driving data, my driving data, and the reverse driving)
def img_generator(samples, batch_size=32, is_validation_generator = False):
    num_samples = len(samples)
    while 1: # Loop forever so the generator never terminates
        for offset in range(0, num_samples, batch_size):
            batch_samples = samples[offset:offset+batch_size]

            images = []
            angles = []

            for batch_sample in batch_samples:
                steering_angle = float(batch_sample[3])

                center_image = cv2.imread(batch_sample[0])
                images.append(cv2.cvtColor(center_image, cv2.COLOR_BGR2RGB))
                angles.append(steering_angle)

                images.append(np.fliplr(center_image))
                angles.append(steering_angle * - 1.0)

                # the validation data shouldn't have the extra left/ right or the reversed images as
                # at testing time only data from central camera is used
                if (is_validation_generator == False and batch_sample[1].split('/')[0] != 'data_recovery') :
                    left_image = cv2.imread(batch_sample[1])
                    right_image = cv2.imread(batch_sample[2])


                    if batch_sample[1].split('/')[0] == 'data_reverse':
                        images.append(cv2.cvtColor(left_image, cv2.COLOR_BGR2RGB))
                        angles.append(steering_angle + 0.005 * steering_angle)

                        images.append(cv2.cvtColor(right_image, cv2.COLOR_BGR2RGB))
                        angles.append(steering_angle - 0.008 * steering_angle)

                    else :
                        images.append(cv2.cvtColor(left_image, cv2.COLOR_BGR2RGB))
                        angles.append(steering_angle + 0.01 * steering_angle)

                        images.append(cv2.cvtColor(right_image, cv2.COLOR_BGR2RGB))
                        angles.append(steering_angle - 0.005 * steering_angle)

            X_train = np.array(images)
            y_train = np.array(angles)
            yield sklearn.utils.shuffle(X_train, y_train)

In [48]:
x = Input(shape=(224,224,3))
# This works, and returns the 10-way softmax we defined above.
y = model(x)
y2 = Dense(2, activation='softmax')(y)

In [ ]:
batch_size = 32
batches = datagen.flow(X_train, y_train, batch_size=batch_size, target_size=(224,224), shuffle=True)
val_batches = gen.flow(X_test, y_test, batch_size=batch_size, target_size=(224,224), shuffle=False)